**SOFT DEADLINE:** `20.03.2022 23:59 msk` 

# [5 points] Part 1. Data cleaning

The task is to clear the text data of the crawled web-pages from different sites. 

It is necessary to ensure that the distribution of the 100 most frequent words includes only meaningful words in english language (not particles, conjunctions, prepositions, numbers, tags, symbols).

Determine the order of operations below and carry out the appropriate cleaning.

1. Remove non-english words
1. Remove html-tags (try to do it with regular expression, or play with beautifulsoap library)
1. Apply lemmatization / stemming
1. Remove stop-words
1. Additional processing - At your own initiative, if this helps to obtain a better distribution

#### Hints

1. To do text processing you may use nltk and re libraries
1. and / or any other libraries on your choise

#### Import libraries

In [1]:
import csv
from bs4 import BeautifulSoup
# BeautifulSoup: This library helps us to get the HTML structure of the page that we want to work with. We can then, use its functions to access specific elements and extract relevant information.

#### Data reading

The dataset for this part can be downloaded here: `https://drive.google.com/file/d/1wLwo83J-ikCCZY2RAoYx8NghaSaQ-lBA/view?usp=sharing`

In [56]:
# Open and Read the content of the CSV file 
data_file =open("web_sites_data.csv",'r')
data_file = csv.reader(data_file)

In [65]:
def remove_tags(html):
  
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
    title=soup.title.get_text()
    print(title)
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
  
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)
  
  
# Print the extracted data


In [66]:
for i,row in enumerate(data_file):
    print(row)
    print("=======")
    print(remove_tags(row[0]))
    if (i>=0): break

['<html>\n<head profile="http://www.w3.org/2005/10/profile">\n<LINK REL="SHORTCUT ICON" href="http://i.bookmooch.com/favicon.ico"> \n<link rel="icon" type="image/png" href="http://i.bookmooch.com/favicon.png">\n<title>Robert Ludlum : Apocalypse Watch</title>\n<meta http-equiv="Content-Type" content="text/html">\n\t\n</head>\n<body bgcolor="#FFFFFF" leftmargin="0" topmargin="0" marginwidth="0" marginheight="0" text="#000000" link="#0000FF" vlink="#0000FF" alink="#FF0000" >\n<basefont face="arial, sans-serif"><font face="arial, sans-serif">\n<table width="100%" height="70" border="0" cellpadding="0" cellspacing="0">\n\t<tr><form action="/search" method="get">\n\t\t<td width="283" colspan="2" rowspan="2" bgcolor="#689A9B">\n\t\t\t<a href="/">\n\t\t\t\t<img src="http://i.bookmooch.com/images/bookmooch_logo.gif" width="283" height="66" border="0" alt="BookMooch logo"></a></td>\n\t\t<td width="675" height="38" colspan="9" align="right" bgcolor="#689A9B" xcolor="#689A9B">\n\t\t\n<table border

In [ ]:
print(soup)

<html>
<head profile="http://www.w3.org/2005/10/profile">
<link href="http://i.bookmooch.com/favicon.ico" rel="SHORTCUT ICON"/>
<link href="http://i.bookmooch.com/favicon.png" rel="icon" type="image/png"/>
<title>Paul Merton : My Struggle</title>
<meta content="text/html" http-equiv="Content-Type"/>
</head>
<body alink="#FF0000" bgcolor="#FFFFFF" leftmargin="0" link="#0000FF" marginheight="0" marginwidth="0" text="#000000" topmargin="0" vlink="#0000FF">
<basefont face="arial, sans-serif"/><font face="arial, sans-serif">
<table border="0" cellpadding="0" cellspacing="0" height="70" width="100%">
<tr><form action="/search" method="get">
<td bgcolor="#689A9B" colspan="2" rowspan="2" width="283">
<a href="/">
<img alt="BookMooch logo" border="0" height="66" src="http://i.bookmooch.com/images/bookmooch_logo.gif" width="283"/></a></td>
<td align="right" bgcolor="#689A9B" colspan="9" height="38" width="675" xcolor="#689A9B">
<table border="0" cellpadding="0" cellspacing="0"><tr>
<td align="ri

In [49]:
for i,row in enumerate (soup):
    print(row)

<html>
<head profile="http://www.w3.org/2005/10/profile">
<link href="http://i.bookmooch.com/favicon.ico" rel="SHORTCUT ICON"/>
<link href="http://i.bookmooch.com/favicon.png" rel="icon" type="image/png"/>
<title>Paul Merton : My Struggle</title>
<meta content="text/html" http-equiv="Content-Type"/>
</head>
<body alink="#FF0000" bgcolor="#FFFFFF" leftmargin="0" link="#0000FF" marginheight="0" marginwidth="0" text="#000000" topmargin="0" vlink="#0000FF">
<basefont face="arial, sans-serif"/><font face="arial, sans-serif">
<table border="0" cellpadding="0" cellspacing="0" height="70" width="100%">
<tr><form action="/search" method="get">
<td bgcolor="#689A9B" colspan="2" rowspan="2" width="283">
<a href="/">
<img alt="BookMooch logo" border="0" height="66" src="http://i.bookmooch.com/images/bookmooch_logo.gif" width="283"/></a></td>
<td align="right" bgcolor="#689A9B" colspan="9" height="38" width="675" xcolor="#689A9B">
<table border="0" cellpadding="0" cellspacing="0"><tr>
<td align="ri

In [50]:
print(soup)

<html>
<head profile="http://www.w3.org/2005/10/profile">
<link href="http://i.bookmooch.com/favicon.ico" rel="SHORTCUT ICON"/>
<link href="http://i.bookmooch.com/favicon.png" rel="icon" type="image/png"/>
<title>Paul Merton : My Struggle</title>
<meta content="text/html" http-equiv="Content-Type"/>
</head>
<body alink="#FF0000" bgcolor="#FFFFFF" leftmargin="0" link="#0000FF" marginheight="0" marginwidth="0" text="#000000" topmargin="0" vlink="#0000FF">
<basefont face="arial, sans-serif"/><font face="arial, sans-serif">
<table border="0" cellpadding="0" cellspacing="0" height="70" width="100%">
<tr><form action="/search" method="get">
<td bgcolor="#689A9B" colspan="2" rowspan="2" width="283">
<a href="/">
<img alt="BookMooch logo" border="0" height="66" src="http://i.bookmooch.com/images/bookmooch_logo.gif" width="283"/></a></td>
<td align="right" bgcolor="#689A9B" colspan="9" height="38" width="675" xcolor="#689A9B">
<table border="0" cellpadding="0" cellspacing="0"><tr>
<td align="ri

#### Data processing

#### Vizualization

As a visualisation, it is necessary to construct a frequency distribution of words (the 100 most common words), sorted by frequency. 

For visualization purposes we advice you to use plotly, but you are free to choose other libraries

#### Provide examples of processed text (some parts)

Is everything all right with the result of cleaning these examples? What kind of information was lost?

# [10 points] Part 2. Duplicates detection. LSH

#### Libraries you can use

1. LSH - https://github.com/ekzhu/datasketch
1. LSH - https://github.com/mattilyra/LSH
1. Any other library on your choise

1. Detect duplicated text (duplicates do not imply a complete word-to-word match, but texts that may contain a paraphrase, rearrangement of words, sentences)
1. Make a plot dependency of duplicates on shingle size (with fixed minhash length) 
1. Make a plot dependency of duplicates on minhash length (with fixed shingle size)

# [Optional 10 points] Part 3. Topic model

In this part you will learn how to do topic modeling with common tools and assess the resulting quality of the models. 

The provided data contain chunked stories by Edgar Allan Poe (EAP), Mary Shelley (MWS), and HP Lovecraft (HPL).

The dataset can be downloaded here: `https://drive.google.com/file/d/14tAjAzHr6UmFVFV7ABTyNHBh-dWHAaLH/view?usp=sharing`

#### Preprocess dataset with the functions from the Part 1

#### Quality estimation

Implement the following three quality fuctions: `coherence` (or `tf-idf coherence`), `normalized PMI`, `based on the distributed word representation`(you can use pretrained w2v vectors or some other model). You are free to use any libraries (for instance gensim) and components.

### Topic modeling

Read and preprocess the dataset, divide it into train and test parts `sklearn.model_selection.train_test_split`. Test part will be used in classification part. For simplicity we do not perform cross-validation here, but you should remember about it.

Plot the histogram of resulting tokens counts in the processed datasets.

Plot the histogram of resulting tokens counts in the processed datasets.

#### NMF

Implement topic modeling with NMF (you can use `sklearn.decomposition.NMF`) and print out resulting topics. Try to change hyperparameters to better fit the dataset.

#### LDA

Implement topic modeling with LDA (you can use gensim implementation) and print out resulting topics. Try to change hyperparameters to better fit the dataset.

### Additive regularization of topic models 

Implement topic modeling with ARTM. You may use bigartm library (simple installation for linux: pip install bigartm) or TopicNet framework (`https://github.com/machine-intelligence-laboratory/TopicNet`)

Create artm topic model fit it to the data. Try to change hyperparameters (number of specific and background topics) to better fit the dataset. Play with smoothing and sparsing coefficients (use grid), try to add decorrelator. Print out resulting topics.

Write a function to convert new documents to topics probabilities vectors.

Calculate the quality scores for each model. Make a barplot to compare the quality.